In [106]:
import torch
import torch.nn.functional as F 
import torch.optim 
from tqdm import tqdm
import numpy as np 

NUM_OF_DAYS_BEFORE = 100

train_data = torch.from_numpy(np.load('BTC-USD/btc-usd-train-data.npy', allow_pickle=True))
train_label = torch.from_numpy(np.load('BTC-USD/btc-usd-train-label.npy', allow_pickle=True))

test_data = torch.from_numpy(np.load('BTC-USD/btc-usd-test-data.npy', allow_pickle=True))
test_label = torch.from_numpy(np.load('BTC-USD/btc-usd-test-label.npy', allow_pickle=True))

print(train_data.shape)

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(4 * NUM_OF_DAYS_BEFORE, 128)
        self.fc2 = torch.nn.Linear(128, 89)
        self.fc3 = torch.nn.Linear(89, 4)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

torch.Size([200, 100, 4])


In [109]:
net = Net()

optimizer = torch.optim.Adam(net.parameters(), lr = 0.001) 

EPOCH = 200

for epoch in tqdm(range(EPOCH)):
    for X, y in zip(train_data, train_label):
        X = X.view(4 * NUM_OF_DAYS_BEFORE)
        y = y.view(4)
        out = net(X)

        loss = F.l1_loss(out, y)

        net.zero_grad()
        loss.backward()
        optimizer.step()
print()
print("Current loss is", loss.item(), "in epoch", epoch + 1)
# torch.save(net, 'xox_net.torch')

100%|██████████| 200/200 [03:05<00:00,  1.08it/s]
Current loss is 340.461669921875 in epoch 200



In [127]:

index = 0
out = net(test_data[index].view(4 * NUM_OF_DAYS_BEFORE))
label = test_label[index]

print("BITCOIN/USD Value Estimator for one day, Deep Neural Network")
print("Real values:")
print(
''' 
    Opening Price: {:.1f} USD
    Closing Price: {:.1f} USD 
    Highest Price: {:.1f} USD 
    Lowest Price:  {:.1f} USD'''.format(label[1].item(), label[0].item(), label[2].item(), label[3].item()))
print()
print("Network prediction:")
print(
''' 
    Opening Price: {:.1f} USD
    Closing Price: {:.1f} USD 
    Highest Price: {:.1f} USD 
    Lowest Price:  {:.1f} USD'''.format(out[1].item(), out[0].item(), out[2].item(), out[3].item()))


# torch.save(net, 'bitcoin.tensor')

BITCOIN/USD Value Estimator for one day, Deep Neural Network
Real values:
 
    Opening Price: 6614.1 USD
    Closing Price: 6662.9 USD 
    Highest Price: 6696.5 USD 
    Lowest Price:  6591.2 USD

Network prediction:
 
    Opening Price: 6563.1 USD
    Closing Price: 6560.1 USD 
    Highest Price: 6573.0 USD 
    Lowest Price:  6493.3 USD
